In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv('Twitter_Data.csv', index_col=False)
df.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [3]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162969 entries, 0 to 162979
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   clean_text  162969 non-null  object 
 1   category    162969 non-null  float64
dtypes: float64(1), object(1)
memory usage: 3.7+ MB


In [4]:
df= df[:int(df.shape[0]*0.2)]
df.shape

(32593, 2)

## Bag of words  DataFrame

In [5]:
cv1 = CountVectorizer(stop_words='english')
df_bow = cv1.fit_transform(df["clean_text"])
df_bow=df_bow.toarray()
df_bow.shape

(32593, 39675)

## Ngrams n=2 dataframe can not make dataset as memory issue 

In [ ]:
# cv1 = CountVectorizer(stop_words='english',ngram_range=(2, 2))
# df_bow = cv1.fit_transform(df["clean_text"])
# df_bow=df_bow.toarray()
# df_bow.shape

In [6]:
xtrain = df_bow[:int(df_bow.shape[0]*0.8)]
xtest = df_bow[int(df_bow.shape[0]*0.8):]

In [7]:
y = np.array(df['category'])
yN =[]
for i in y:
    if(i == 1.0):
        yN.append([1,0,0])
    elif(i==0.0):
        yN.append([0,1,0])
    else:
        yN.append([0,0,1])
yN[:5]    

[[0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0]]

In [8]:
ytrain = np.array(yN[:int(df_bow.shape[0]*0.8)])
ytest = np.array(yN[int(df_bow.shape[0]*0.8):])
ytrain = np.reshape(ytrain,(ytrain.shape[0],3))
ytest = np.reshape(ytest,(ytest.shape[0],3))
print(ytrain.shape, ytest.shape)

(26074, 3) (6519, 3)


In [9]:
x = tf.constant(xtrain, dtype=tf.float32)
w=tf.Variable(tf.random.uniform(shape=[3,x.shape[1]], minval=0,maxval=15, dtype=tf.float32, seed=123456789))

In [ ]:
opti= Adam(learning_rate=1e-1)
iter =200
c=[]
ytrain=tf.constant(ytrain,dtype=tf.float32)
loss=tf.keras.losses.CategoricalCrossentropy(reduction=tf.keras.losses.Reduction.SUM)
for i in range(iter):
    with tf.GradientTape() as tape:
        yhat = tf.nn.softmax(tf.matmul(x,tf.transpose(w)),axis=1)
        cost = loss(yhat, ytrain)
        
    c.append(cost)
    grad = tape.gradient(cost,[w])
    opti.apply_gradients(zip(grad,[w])) # adma optimizer

In [ ]:
np.savetxt("SoftmaxWeights_BOW.csv", w, delimiter=",")

In [ ]:
l=np.arange(1,len(c)+1)
plt.plot(l, c, label = "line 2")
  
# naming the x axis -->
plt.xlabel('iteration')
# naming the y axis
plt.ylabel('cost')
# giving a title to my graph
plt.title('Cost function Recommendor system!')
  
# show a legend on the plot
plt.legend()
  
# function to show the plot
plt.show()

In [ ]:
xtest = tf.constant(xtest, dtype=tf.float32)
yhatN = tf.nn.softmax(tf.matmul(xtest,tf.transpose(w)),axis=1).numpy()
yhat = []
for i in yhatN:
    a = np.argmax(i)
    if(a==0):
        yhat.append(1.0)
    elif(a==1):
        yhat.append(0.0)
    else:
        yhat.append(-1.0)

In [ ]:
ytN=np.array(y[int(df_bow.shape[0]*0.8):])
ytest = np.array(ytN)
ytest = np.reshape((ytest),(ytest.shape[0],1))
ytest.shape

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
## Accuracy
yhat = np.array(yhat)
yhat=np.reshape((yhat),(yhat.shape[0],1))
print(yhat.shape)
print(accuracy_score(ytest, yhat))
print(f1_score(ytest, yhat, average='micro'))
